### Import Libraries

In [159]:
# Import twitter dependencies
import tweepy
from config import *

In [160]:
# Import transformation dependencies
import pandas as pd
from datetime import datetime, date
import json
import re 

In [161]:
# For sentiment and subjectivity analysis
from textblob import TextBlob

# Tweepy Setup

In [162]:
# Tweepy Setup
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [185]:
# Tweepy search parameters
search_terms = ['covid+vaccine OR covid-19+vaccine OR pfizer OR pfizer-biontech OR moderna OR astrazeneca OR astra+zeneca -filter:retweets']
end_date = date(2021,3,11)
max_id ='1370158327167123457'
limit = 900 # this is my 15 minute limit :'( breaks if this becomes 1000

# Save files
output_csv ='csvs/tw35.csv'
output_json ='json/raw35.json'

# Functions

Collect and save data

In [175]:
# Pulls data from api and returns 2 lists:
    # one which is parsed for selected variables
    # one with all the raw data

data = []
raw = []
tweets = tweepy.Cursor(api.search, location='canada', max_id = max_id, place='canada', lang ='en', q=search_terms, tweet_mode='extended').items(limit)
for t in tweets: 
    tweet ={
        'created' :t.created_at,
        'id_str' : t.id_str,
        'text' : t.full_text,
        'user_id' :t.user.id,
        'screen_name': t.user.screen_name,
        'location': t.user.location,
        'followers_count': t.user.followers_count,
        'user_favourites_count': t.user.favourites_count,
        'time_zone' :t.user.time_zone,
        'geo_enabled' :t.user.geo_enabled,
        'verified' :t.user.verified,
        'status_count': t.user.statuses_count,
        'geo':t.geo,
        'coords':t.coordinates,
        'retweet_count': t.retweet_count,
        'tweet_favourite_count' : t.favorite_count
    }
    # add sorted data to one list 
    data.append(tweet)
    # keep raw data in case I realize later that I've removed something important 
    raw.append(t)
print('done!')

done!


In [176]:
# Add parsed data to dataframe, convert to csv
tweets_df = pd.DataFrame(data)

# Save raw data to json 
with open(output_json, 'w') as file:
    for resp in raw:
        jso = json.dumps(resp._json,sort_keys=True, indent=3)
        file.write(jso)
tweets_df.tail(3)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count
897,2021-03-11 23:42:56,1370158330459664388,✅ January 20th: Inaugurated.\n\n✅ March 2nd: O...,14193665,joshiku,,926,28534,None,True,False,17064,None,None,0,0
898,2021-03-11 23:42:56,1370158330358947842,Kroger pharmacy gives wrong shot to customers ...,62876792,mdp4202,"Frumunda, Upper Wazoo",5125,19634,None,False,False,283393,None,None,1,1
899,2021-03-11 23:42:56,1370158327167123457,RT @joshtpm: Biden Set To Announce All Adults ...,47632343,sean_mpls,"Minneapolis, MN",596,2903,None,True,False,15077,None,None,54,0


### Clean tweets

In [177]:
# not working when I put these into a function
tweets_df = tweets_df.drop_duplicates(subset=['id_str'])
tweets_df = tweets_df.dropna(subset=['id_str', 'screen_name'])

In [178]:
# Makes tweets more readable 
def CleanTweets(txt): 
    # remove @ username 
    # txt = re.sub(r'@[A-Za-z0-9)]+','', txt)

    # remove @ from username
    txt = txt.replace('@','')    
    
    # remove RT    
    txt = re.sub(r'RT[\s]+','',txt)
    
    # remove # but leave txt
    txt = re.sub(r'#','', txt)
    
    # remove hyperlinks
    txt = re.sub(r'https?:\/\/\S+', '', txt)
    return txt

In [179]:
# Clean the text field 
tweets_df['text'] = tweets_df['text'].apply(CleanTweets)

In [180]:
# to get subjectivity 
def RateSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

In [181]:
# to get polarity
def RatePolarity(txt):
    return TextBlob(txt).sentiment.polarity

In [182]:
# add worded sentiment based on polarity score
def GetSentiment(num):
    if num < 0: 
        return 'negative' 
    elif num == 0:
        return 'neutral'
    else:
        return 'positive'

In [183]:
# Create Columns
def RateTweets(df):
    df['Subjectivity'] = df['text'].apply(RateSubjectivity)
    df['Polarity'] = df['text'].apply(RatePolarity)
    df['Sentiment'] = df['Polarity'].apply(GetSentiment)
    return df 

In [184]:
tweets_df = RateTweets(tweets_df)
tweets_df.to_csv(output_csv, index=False, encoding='UTF-8')
tweets_df.tail(3)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count,Subjectivity,Polarity,Sentiment
897,2021-03-11 23:42:56,1370158330459664388,✅ January 20th: Inaugurated.\n\n✅ March 2nd: O...,14193665,joshiku,,926,28534,None,True,False,17064,None,None,0,0,0.166667,0.0,neutral
898,2021-03-11 23:42:56,1370158330358947842,Kroger pharmacy gives wrong shot to customers ...,62876792,mdp4202,"Frumunda, Upper Wazoo",5125,19634,None,False,False,283393,None,None,1,1,0.650000,-0.3,negative
899,2021-03-11 23:42:56,1370158327167123457,joshtpm: Biden Set To Announce All Adults Elig...,47632343,sean_mpls,"Minneapolis, MN",596,2903,None,True,False,15077,None,None,54,0,0.000000,0.0,neutral


# SQLAlchemy
### Connecting to a certain other host 

In [5]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [12]:
engine = create_engine(conn, echo=False)

In [13]:
Base = automap_base()
Base.prepare(engine, reflect=True)  

In [14]:
# Test connection
# it works!!!
# not anymore
# kill kernal if conn is wrong

Base.classes.keys()

['tw2']

### Don't run yet
Clean up your tweets! 

In [9]:
_df = pd.read_csv('csvs/tw2.csv')
_df.to_sql(name='tw2', con=engine, if_exists='append', index=True)
print('added!')

added!


In [ ]:
# session things
session = Session(engine)
session.close()
